In [1]:
### Example taken from :: https://machinelearningmastery.com/handle-missing-data-python/

1. Pima Indians Diabetes Dataset


The Pima Indians Diabetes Dataset involves predicting the onset of diabetes within 5 years in Pima Indians given medical details.

It is a binary (2-class) classification problem. The number of observations for each class is not balanced. There are 768 observations with 8 input variables and 1 output variable. The variable names are as follows:

0. Number of times pregnant.
1. Plasma glucose concentration a 2 hours in an oral glucose tolerance test.
2. Diastolic blood pressure (mm Hg).
3. Triceps skinfold thickness (mm).
4. 2-Hour serum insulin (mu U/ml).
5. Body mass index (weight in kg/(height in m)^2).
6. Diabetes pedigree function.
7. Age (years).
8. Class variable (0 or 1).

In [2]:
import pandas as pd
dataset = pd.read_csv('pima-indians-diabetes.data.txt', header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
print(dataset.describe())

                0           1           2           3           4           5  \
count  768.000000  768.000000  768.000000  768.000000  768.000000  768.000000   
mean     3.845052  120.894531   69.105469   20.536458   79.799479   31.992578   
std      3.369578   31.972618   19.355807   15.952218  115.244002    7.884160   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      1.000000   99.000000   62.000000    0.000000    0.000000   27.300000   
50%      3.000000  117.000000   72.000000   23.000000   30.500000   32.000000   
75%      6.000000  140.250000   80.000000   32.000000  127.250000   36.600000   
max     17.000000  199.000000  122.000000   99.000000  846.000000   67.100000   

                6           7           8  
count  768.000000  768.000000  768.000000  
mean     0.471876   33.240885    0.348958  
std      0.331329   11.760232    0.476951  
min      0.078000   21.000000    0.000000  
25%      0.243750   24.000000    0.000000  
50%   

This is useful.

We can see that there are columns that have a minimum value of zero (0). On some columns, a value of zero does not make sense and indicates an invalid or missing value.

Specifically, the following columns have an invalid zero minimum value:

1: Plasma glucose concentration
2: Diastolic blood pressure
3: Triceps skinfold thickness
4: 2-Hour serum insulin
5: Body mass index

######################################################################################################
We can get a count of the number of missing values on each of these columns. We can do this my marking all of the values in the subset of the DataFrame we are interested in that have zero values as True. We can then count the number of true values in each column.

We can do this my marking all of the values in the subset of the DataFrame we are interested in that have zero values as True. We can then count the number of true values in each column.

In [4]:
print((dataset[[1,2,3,4,5]]== 0).sum())

1      5
2     35
3    227
4    374
5     11
dtype: int64


In Python, specifically Pandas, NumPy and Scikit-Learn, we mark missing values as NaN.

Values with a NaN value are ignored from operations like sum, count, etc.

We can mark values as NaN easily with the Pandas DataFrame by using the replace() function on a subset of the columns we are interested in.

After we have marked the missing values, we can use the isnull() function to mark all of the NaN values in the dataset as True and get a count of the missing values for each column.

In [6]:
import numpy as np
# mark zero values as missing or NaN
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, np.NaN )

# count the number of NaN values in each column
print(dataset.isnull().sum())


0      0
1      5
2     35
3    227
4    374
5     11
6      0
7      0
8      0
dtype: int64


3. Missing Values Causes Problems

Having missing values in a dataset can cause errors with some machine learning algorithms.

In this section, we will try to evaluate a the Linear Discriminant Analysis (LDA) algorithm on the dataset with missing values.

This is an algorithm that does not work when there are missing values in the dataset.

The below example marks the missing values in the dataset, as we did in the previous section, then attempts to evaluate LDA using 3-fold cross validation and print the mean accuracy.

In [8]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# split dataset into inputs and outputs
values = dataset.values   ## generated matrix out of the dataframe
X = values[:,:8]
y = values[:,8]
# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()

kfold = KFold(n_splits =3 , random_state = 7)
result = cross_val_score(model,X, y, cv= kfold, scoring = 'accuracy' )
print('result.mean():', result.mean())


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

This is as we expect.

We are prevented from evaluating an LDA algorithm (and other algorithms) on the dataset with missing values.

Now, we can look at methods to handle the missing values.

#################################################################
4. Remove Rows With Missing Values

The simplest strategy for handling missing data is to remove records that contain a missing value.

We can do this by creating a new Pandas DataFrame with the rows containing missing values removed.

Pandas provides the dropna() function that can be used to drop either columns or rows with missing data. We can use dropna() to remove all rows with missing data, as follows:

In [10]:
# drop rows with missing values
dataset.dropna(inplace = True)
# summarize the number of rows and columns in the dataset
print(dataset.shape)

(392, 9)


We now have a dataset that we could use to evaluate an algorithm sensitive to missing values like LDA.

In [11]:
values = dataset.values   ## generated matrix out of the dataframe
X = values[:,:8]
y = values[:,8]
# evaluate an LDA model on the dataset using k-fold cross validation
model = LinearDiscriminantAnalysis()

kfold = KFold(n_splits =3 , random_state = 7)
result = cross_val_score(model,X, y, cv= kfold, scoring = 'accuracy' )
print('result.mean():', result.mean())

result.mean(): 0.78582892934


Removing rows with missing values can be too limiting on some predictive modeling problems, an alternative is to impute missing values.


In [13]:
dataset = pd.read_csv('pima-indians-diabetes.data.txt', header=None)
# mark zero values as missing or NaN
dataset[[1,2,3,4,5]] = dataset[[1,2,3,4,5]].replace(0, np.NaN)

# fill missing values with mean column values
dataset.fillna(dataset.mean(), inplace = True)

# count the number of NaN values in each column
print(dataset.isnull().sum())


0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64


**The scikit-learn library provides the Imputer() pre-processing class that can be used to replace missing values.

It is a flexible class that allows you to specify the value to replace (it can be something other than NaN) and the technique used to replace it (such as mean, median, or mode). The Imputer class operates directly on the NumPy array instead of the DataFrame.

The example below uses the Imputer class to replace missing values with the mean of each column then prints the number of NaN values in the transformed matrix. **

In [18]:
from sklearn.preprocessing import Imputer
#values = dataset.values   ## generated matrix out of the dataframe
#X = values[:,:8]
#y = values[:,8]
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis =0)
imp.fit(dataset)   ######## working
print(dataset.isnull().sum())  ###### working

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64


In [27]:
import pandas as pd
df = pd.read_csv('pima-indians-diabetes.data.txt', header=None)

y = df[8]
X = df.drop(8, axis =1)

#print((X[[1,2,3,4,5]]== 0).sum())   # works
print((X == 0).sum()) # also works
imp = Imputer(missing_values = 0, strategy = 'mean', axis =0)
imp.fit(X)   ######## working
print((X == 0).sum())  ###### working
X.head()

0    111
1      5
2     35
3    227
4    374
5     11
6      0
7      0
dtype: int64
0    111
1      5
2     35
3    227
4    374
5     11
6      0
7      0
dtype: int64


,0,1,2,3,4,5,6,7
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [34]:
import pandas as pd
df = pd.read_csv('pima-indians-diabetes.data.txt', header=None)

y = df[8]
X = df.drop(8, axis =1)

print((X == 0).sum()) # also works
imp = Imputer(missing_values = 0, strategy = 'mean', axis =0)
X_imp = imp.fit_transform(X)
print((X_imp == 0).sum())
X_imp.head()

0    111
1      5
2     35
3    227
4    374
5     11
6      0
7      0
dtype: int64
0


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
'''
https://stackoverflow.com/questions/33660836/impute-entire-dataframe-all-columns-using-scikit-learn-sklearn-without-itera
If you want the mean or median you could do something like:

fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=1)
imputed_DF = pd.DataFrame(fill_NaN.fit_transform(DF))
imputed_DF.columns = DF.columns
imputed_DF.index = DF.index
If you want to fill them with 0s or something you could always just do:

DF[DF.isnull()] = 0
'''

In [40]:
import pandas as pd
df = pd.read_csv('pima-indians-diabetes.data.txt', header=None)

#values = df.values
#X = values[:,0:8]
#y = values[:,8]
y = df[8]
X = df.drop(8, axis =1)
print(X)


      0    1   2   3    4     5      6   7
0     6  148  72  35    0  33.6  0.627  50
1     1   85  66  29    0  26.6  0.351  31
2     8  183  64   0    0  23.3  0.672  32
3     1   89  66  23   94  28.1  0.167  21
4     0  137  40  35  168  43.1  2.288  33
5     5  116  74   0    0  25.6  0.201  30
6     3   78  50  32   88  31.0  0.248  26
7    10  115   0   0    0  35.3  0.134  29
8     2  197  70  45  543  30.5  0.158  53
9     8  125  96   0    0   0.0  0.232  54
10    4  110  92   0    0  37.6  0.191  30
11   10  168  74   0    0  38.0  0.537  34
12   10  139  80   0    0  27.1  1.441  57
13    1  189  60  23  846  30.1  0.398  59
14    5  166  72  19  175  25.8  0.587  51
15    7  100   0   0    0  30.0  0.484  32
16    0  118  84  47  230  45.8  0.551  31
17    7  107  74   0    0  29.6  0.254  31
18    1  103  30  38   83  43.3  0.183  33
19    1  115  70  30   96  34.6  0.529  32
20    3  126  88  41  235  39.3  0.704  27
21    8   99  84   0    0  35.4  0.388  50
22    7  19

In [42]:
imp = Imputer(missing_values = 0, strategy = 'mean', axis =0)
X_imp = pd.DataFrame(imp.fit_transform(X))
X_imp.columns = X.columns
X_imp.index = X.index
print((X_imp == 0).sum())
X_imp

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64


,0,1,2,3,4,5,6,7
0,6.000000,148.0,72.000000,35.00000,155.548223,33.600000,0.627,50.0
1,1.000000,85.0,66.000000,29.00000,155.548223,26.600000,0.351,31.0
2,8.000000,183.0,64.000000,29.15342,155.548223,23.300000,0.672,32.0
3,1.000000,89.0,66.000000,23.00000,94.000000,28.100000,0.167,21.0
4,4.494673,137.0,40.000000,35.00000,168.000000,43.100000,2.288,33.0
5,5.000000,116.0,74.000000,29.15342,155.548223,25.600000,0.201,30.0
6,3.000000,78.0,50.000000,32.00000,88.000000,31.000000,0.248,26.0
7,10.000000,115.0,72.405184,29.15342,155.548223,35.300000,0.134,29.0
8,2.000000,197.0,70.000000,45.00000,543.000000,30.500000,0.158,53.0
9,8.000000,125.0,96.000000,29.15342,155.548223,32.457464,0.232,54.0
